In [ ]:
import numpy as np
import pandas as pd
!pip install transformers
import transformers

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
train_path = "/content/drive/Shareddrives/ML Modelling/data/train/13-Feb-76k_Data_6digit.csv"
df = pd.read_csv(train_path)

In [ ]:
df.head()

In [ ]:
df.keys()

In [ ]:
df=df.drop(['Date', 'Sr No', 'Parent-Product','Child - Product (Other)', 'SubChild (Yarn & Thread)', 'Material','Gender'],axis=1)

In [ ]:
df.head()

In [ ]:
value_counts = df['6 Digit Code'].value_counts()
mask = value_counts >= 20
# Use the mask to index into the original DataFrame and select the repeating rows
df1 = df[df['6 Digit Code'].isin(value_counts[mask].index)]

In [ ]:
df1 = df1.reset_index(drop=True)

In [ ]:
from sklearn import preprocessing
label_enc = preprocessing.LabelEncoder()
df1['Label6Digit']=label_enc.fit_transform(df1['6 Digit Code'])

In [ ]:
texts = []
texts = df1['PRODUCTS']
labels = df1['Label6Digit'].values

In [ ]:
print(texts)

In [ ]:
print(np.unique(df['6 Digit Code']))
num_labels = len(set(labels))
print(num_labels)

[210690 320910 321290 330300 330410 330420 330491 330499 330510 330530
 330590 340130 340600 350610 392049 392051 392310 392330 392410 392490
 392690 401519 420212 420219 420221 420222 420231 420232 420239 420292
 420299 420310 420330 420340 442191 442199 470200 481141 481710 481720
 481930 482010 482090 490110 490199 500600 510910 510990 520420 520710
 520790 520819 520839 520851 520852 520859 540600 540774 570250 570291
 600410 600622 610190 610322 610329 610331 610332 610339 610342 610349
 610419 610422 610429 610431 610439 610441 610442 610444 610449 610451
 610452 610453 610462 610463 610469 610510 610590 610610 610620 610690
 610711 610829 610910 610990 611011 611020 611030 611090 611211 611219
 611239 611249 611420 611490 611510 611529 611594 611595 611599 611710
 611780 620190 620220 620240 620290 620322 620329 620331 620332 620339
 620341 620342 620343 620349 620412 620419 620422 620429 620431 620432
 620433 620439 620441 620442 620443 620444 620449 620451 620452 620453
 62045

In [ ]:
df.shape,df1.shape

((76933, 5), (76933, 6))

In [ ]:
df1.head()

,PRODUCTS,COMMODITY CODE,Label,6 Digit Code,Label6,Label6Digit
0,"!M?Erfect Women's Mini Dress S Blue, Blend - c...",6204420090,153,620442,133,133
1,"""Glow thanks to Joe"" Body Scrub",3304990000,7,330499,7,7
2,"& Denim Women's Jacket 20 Red, 100% - cotton",6204329090,148,620432,129,129
3,"& Other Stories Cardigan S Brown, Blend - cotton",6110209900,106,611020,95,95
4,"& Other Stories Cardigan S Green, Blend - cotton",6110209900,106,611020,95,95


In [ ]:
text_lengths = [len(str(texts[i]).split()) for i in range(len(texts))]
print(min(text_lengths))
print(max(text_lengths))

1
39


In [ ]:
sum([1 for i in range(len(text_lengths)) if text_lengths[i] >= 2])

76917

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
print('Original Text: ', texts[0], '\n')
print('Tokenized Text: ', tokenizer.tokenize(texts[0]), '\n')
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))

Original Text:  !M?Erfect Women's Mini Dress S Blue, Blend - cotton, polyester 

Tokenized Text:  ['!', 'm', '?', 'er', '##fect', 'women', "'", 's', 'mini', 'dress', 's', 'blue', ',', 'ble', '##nd', '-', 'cotton', ',', 'pol', '##yes', '##ter'] 

Token IDs:  [106, 155, 136, 10162, 90349, 11847, 112, 161, 18584, 46836, 161, 12442, 117, 10803, 11408, 118, 33991, 117, 14841, 25538, 11048]


In [ ]:
from transformers import AdamW, BertConfig
num_labels = len(set(labels))
print(num_labels)

261


In [ ]:
model = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', num_labels=num_labels,
                                                            output_attentions=False, output_hidden_states=False)

In [ ]:
##### Text IDs
text_ids = [tokenizer.encode(text, max_length= 100, pad_to_max_length=True) for text in texts]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
#### ----Mask attributes
att_masks = []
for ids in text_ids:
    masks = [int(id > 0) for id in ids]
    att_masks.append(masks)

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_val_x, train_y, test_val_y = train_test_split(text_ids, labels, random_state=111, test_size=0.30)
train_m, test_val_m = train_test_split(att_masks, random_state=111, test_size=0.30)

test_x, val_x, test_y, val_y = train_test_split(test_val_x, test_val_y, random_state=111, test_size=0.50)
test_m, val_m = train_test_split(test_val_m, random_state=111, test_size=0.50)

In [ ]:
print('train_x=',len(train_x))
print('test_val_x=',len(test_val_x))

print('train_y=',len(train_y))
print('test_val_y=',len(test_val_y))

print('train_m=',len(train_m))
print('test_val_m=',len(test_val_m))
print('-------------------------------')
print('test_x=',len(test_x))
print('val_x=',len(val_x))

print('test_y=',len(test_y))
print('val_y=',len(val_y))

print('test_m=',len(test_m))
print('val_m=',len(val_m))

train_x= 53853
test_val_x= 23080
train_y= 53853
test_val_y= 23080
train_m= 53853
test_val_m= 23080
-------------------------------
test_x= 11540
val_x= 11540
test_y= 11540
val_y= 11540
test_m= 11540
val_m= 11540


In [ ]:
##### ---- Convert into Tensors----------
import torch

train_x = torch.tensor(train_x)
test_x = torch.tensor(test_x)
val_x = torch.tensor(val_x)
train_y = torch.tensor(train_y)
test_y = torch.tensor(test_y)
val_y = torch.tensor(val_y)
train_m = torch.tensor(train_m)
test_m = torch.tensor(test_m)
val_m = torch.tensor(val_m)

print(train_x.shape)
print(test_x.shape)
print(val_x.shape)
print(train_y.shape)
print(test_y.shape)
print(val_y.shape)
print(train_m.shape)
print(test_m.shape)
print(val_m.shape)

torch.Size([53853, 100])
torch.Size([11540, 100])
torch.Size([11540, 100])
torch.Size([53853])
torch.Size([11540])
torch.Size([11540])
torch.Size([53853, 100])
torch.Size([11540, 100])
torch.Size([11540, 100])


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

train_data = TensorDataset(train_x, train_m, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_x, val_m, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
num_labels = len(set(labels))
print(num_labels)
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=num_labels,
                                                            output_attentions=False, output_hidden_states=False)

261


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
##### ------------Set CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [ ]:

# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (105879, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
[n for n, p in model.named_parameters()]

learning_rate = 3e-5
adam_epsilon = 1e-8

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.2},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

num_epochs = 15
total_steps = len(train_dataloader) * num_epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


##### ---------- Transformers --Time

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import numpy as np
import random

seed_val = 111

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
train_losses = []
val_losses = []
num_mb_train = len(train_dataloader)
num_mb_val = len(val_dataloader)

print(num_mb_train, num_mb_val)
if num_mb_val == 0:
    num_mb_val = 1

for n in range(num_epochs):
    train_loss = 0
    val_loss = 0
    start_time = time.time()
    
    for k, (mb_x, mb_m, mb_y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        model.train()
        

        #print( mb_x, mb_m, mb_y)
        mb_x = mb_x.to(device)
        mb_m = mb_m.to(device)
        mb_y = mb_y.to(device)
        
        outputs = model(mb_x, attention_mask=mb_m, labels=mb_y)
        
        loss = outputs[0]
        #loss = model_loss(outputs[1], mb_y)
        print(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss += loss.data / num_mb_train
    
        print("\nTrain loss after itaration %i: %f" % (n+1, train_loss))
        # print("\nTrain loss after itaration %i: %f" % (n+1, train_loss.item()))
    train_losses.append(train_loss.cpu())
    
    with torch.no_grad():
        model.eval()
        
        for k, (mb_x, mb_m, mb_y) in enumerate(val_dataloader):
            mb_x = mb_x.to(device)
            mb_m = mb_m.to(device)
            mb_y = mb_y.to(device)
        
            outputs = model(mb_x, attention_mask=mb_m, labels=mb_y)
            
            loss = outputs[0]
            #loss = model_loss(outputs[1], mb_y)
            
            val_loss += loss.data / num_mb_val
            
            print("Validation loss after itaration %i: %f" % (n+1, val_loss))
            # print("Validation loss after iteration %i: %f" % (n+1, val_loss.item()))
        val_losses.append(val_loss.cpu())
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Time: {epoch_mins}m {epoch_secs}s')

Streaming output truncated to the last 5000 lines.
Train loss after itaration 2: 0.098973
0.003937168512493372

Train loss after itaration 2: 0.098973
0.0007665548473596573

Train loss after itaration 2: 0.098973
2.4687628746032715

Train loss after itaration 2: 0.099065
0.025302769616246223

Train loss after itaration 2: 0.099066
1.0578800439834595

Train loss after itaration 2: 0.099105
0.00047790620010346174

Train loss after itaration 2: 0.099105
2.397418260574341

Train loss after itaration 2: 0.099194
0.0006639536004513502

Train loss after itaration 2: 0.099194
0.0011125714518129826

Train loss after itaration 2: 0.099194
0.00517698097974062

Train loss after itaration 2: 0.099194
0.0031573595479130745

Train loss after itaration 2: 0.099194
0.002597709186375141

Train loss after itaration 2: 0.099195
0.008885250426828861

Train loss after itaration 2: 0.099195
0.006566289812326431

Train loss after itaration 2: 0.099195
0.001018534996546805

Train loss after itaration 2: 0.0991

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.plot(val_losses)
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

NameError: ignored

In [ ]:
model_save_name = '06-02-2023-Modelling_315_classes_bert-base-multilingual-uncased_'+str(batch_size)+'_'+str(num_epochs)+'_'
path = F"/content/drive/MyDrive/Seers/Trained-Models//{model_save_name}" 
torch.save(model.state_dict(), path)